In [3]:
import sys
sys.path.append('/home/hongn/sapiens/pretrain')
sys.path.append('/home/hongn/sapiens/pretrain/demo')
from extract_feature import *

import cv2
import mmcv
import mmengine
import numpy as np
from tqdm import tqdm
import warnings

import torch
from timesformer.models.vit import TimePSformer, TimeSformer

model = TimePSformer(img_size=224, num_classes=400, num_frames=8, attention_type='time_only')
# model = TimeSformer(img_size=224, num_classes=400, num_frames=8, attention_type='divided_space_time')

dummy_video = torch.randn(2, 3, 8, 224, 224) # (batch x channels x frames x height x width)

pred = model(dummy_video,) # (2, 400)

Loads checkpoint by local backend from path: /home/hongn/sapiens/pretrain/checkpoints/sapiens_0.3b/sapiens_0.3b_epoch_1600_clean.pth on server: t1v-n-3c3909ab-w-0
Done: Loaded checkpoint from /home/hongn/sapiens/pretrain/checkpoints/sapiens_0.3b/sapiens_0.3b_epoch_1600_clean.pth on server: t1v-n-3c3909ab-w-0
The model and loaded state dict do not match exactly

unexpected key in source state_dict: cls_token, pos_embed, patch_embed.projection.weight, patch_embed.projection.bias, layers.0.ln1.weight, layers.0.ln1.bias, layers.0.attn.qkv.weight, layers.0.attn.qkv.bias, layers.0.attn.proj.weight, layers.0.attn.proj.bias, layers.0.ln2.weight, layers.0.ln2.bias, layers.0.ffn.layers.0.0.weight, layers.0.ffn.layers.0.0.bias, layers.0.ffn.layers.1.weight, layers.0.ffn.layers.1.bias, layers.1.ln1.weight, layers.1.ln1.bias, layers.1.attn.qkv.weight, layers.1.attn.qkv.bias, layers.1.attn.proj.weight, layers.1.attn.proj.bias, layers.1.ln2.weight, layers.1.ln2.bias, layers.1.ffn.layers.0.0.weight, l

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1024 but got size 768 for tensor number 1 in the list.

### Get train val test labels csv

In [39]:
import pandas as pd
import glob
phase = 'test'
train = pd.read_csv(f'/mnt/disks/persist2/kinetics400/annotations/{phase}.csv')
id = train.youtube_id.to_list()
train_path = glob.glob(f'/mnt/disks/persist2/kinetics400/{phase}/*.mp4')
print(len(train), len(train_path))

39805 38685


In [40]:
labels = []
for i in range(len(train_path)):
    try:
        idx = id.index(train_path[i].split('/')[-1][:-18])
        labels.append(train.label[idx])
    except:
        print(f"{train_path[i].split('/')[-1][:-18]} is not in the list")

In [41]:
from sklearn import preprocessing 

data4timesformer = pd.DataFrame(
    {'path': train_path,
     'labels': labels,
    })

if phase == 'train':
    label_encoder = preprocessing.LabelEncoder() 
    data4timesformer['labels']= label_encoder.fit_transform(data4timesformer['labels']) 
else:
    data4timesformer['labels']= label_encoder.transform(data4timesformer['labels']) 

data4timesformer.to_csv(f'{phase}.csv', index=False, header=False)  

Modify csv file because missing data

In [ ]:
phase = 'train'
data = pd.read_csv(f'{phase}.csv', names=['paths', 'labels'])
data.head()

if phase == 'train':
    data = data.drop(data.iloc[:,0].to_list().index('/mnt/disks/persist2/kinetics400/train/mf21EBmUtno_000151_000161.mp4'))

144375

In [35]:
data.to_csv(f'{phase}.csv', index=False, header=False)  

In [ ]:
!sudo cp /home/hongn/TimePSFormer/train.csv /mnt/disks/persist2/kinetics400/train.csv

In [ ]:
# phase = 'train'
# data = pd.read_csv(f'{phase}.csv')
# data['labels'].unique() 

# from sklearn import preprocessing 
# label_encoder = preprocessing.LabelEncoder() 
# # Encode labels in column 'species'. 
# data['labels']= label_encoder.fit_transform(data['labels']) 
# data['labels'].unique() 
# data.to_csv(f'{phase}.csv', index=False, header=False)  

In [46]:
phase = 'train'
data = pd.read_csv(f'{phase}.csv')
# data.to_csv(f'{phase}.csv', index=False, header=False)  
data.head()

,/mnt/disks/persist2/kinetics400/train/8mbN0ubh0q8_000000_000010.mp4,101
0,/mnt/disks/persist2/kinetics400/train/9bfc-7If...,17
1,/mnt/disks/persist2/kinetics400/train/VV0KA_f-...,167
2,/mnt/disks/persist2/kinetics400/train/sWCnxZHX...,318
3,/mnt/disks/persist2/kinetics400/train/Am6c9pVi...,356
4,/mnt/disks/persist2/kinetics400/train/pC1hX1cP...,231


In [8]:
data.iloc[:,0].to_list().index('/mnt/disks/persist2/kinetics400/train/mf21EBmUtno_000151_000161.mp4')

144374

In [50]:
data.iloc[144375,1]

np.int64(162)

In [ ]:
sudo mv /mnt/disks/persist2/kinetics400/test.csv /home/hongn/TimePSFormer/test.csv

In [1]:
224//16

14

In [ ]:
sudo cp /home/hongn/TimePSFormer/val.csv /mnt/disks/persist2/kinetics400/val.csv
sudo cp /home/hongn/TimePSFormer/train.csv /mnt/disks/persist2/kinetics400/train.csv
sudo cp /home/hongn/TimePSFormer/test.csv /mnt/disks/persist2/kinetics400/test.csv

In [4]:
import torch.nn as nn
# pool of square window of size=3, stride=2
stride = 64//14
kernelsize = 64-(14-1)*stride
m = nn.MaxPool2d(kernelsize, stride=stride)
# pool of non-square window
m = nn.MaxPool2d((kernelsize, kernelsize), stride=(stride, stride))
input = torch.randn(20, 16, 64, 64)
output = m(input)

In [ ]:
14*14

196

In [5]:
output.shape

torch.Size([20, 16, 14, 14])

In [ ]:
201326592/8192/8/3

In [ ]:
201326592/8/3/2/12

In [ ]:
import pandas as pd